<a href="https://colab.research.google.com/github/rahulombale/swing_strategy_rahul/blob/main/Moving_Average_Strategy_Daily_Opportunity_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# --- 1. SETUP AND INSTALLATION ---
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from datetime import datetime
import os
import warnings
import numpy as np # Ensure numpy is imported
warnings.filterwarnings('ignore')

# --- 2. CONFIGURATION ---

# Mount Google Drive
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"Could not mount drive: {e}")

# This will be updated daily in a real-world scenario
# DATA_FOLDER_DATE = datetime.today().strftime('%Y-%m-%d')
DATA_FOLDER_DATE = "2025-06-17"
DATA_BASE_PATH = '/content/drive/My Drive/stock_data/'
# Path for reading input data - assuming it's in a folder with the current date
INPUT_DATA_PATH = f'{DATA_BASE_PATH}{DATA_FOLDER_DATE}/'
SOURCE_FILE_V40 = 'v40_token.csv'

# Create a dedicated, date-stamped folder for this strategy's results
RESULTS_ROOT = os.path.join(DATA_BASE_PATH, 'MA_Crossover_Result')
DATE_SPECIFIC_RESULTS_DIR = os.path.join(RESULTS_ROOT, DATA_FOLDER_DATE)
OUTPUT_CSV_FILE = f'ma_opportunities_{DATA_FOLDER_DATE}.csv'
BUY_PLOTS_DIR = os.path.join(DATE_SPECIFIC_RESULTS_DIR, 'buy_plots')
SELL_PLOTS_DIR = os.path.join(DATE_SPECIFIC_RESULTS_DIR, 'sell_plots')

# Create output directories if they don't exist
os.makedirs(BUY_PLOTS_DIR, exist_ok=True)
os.makedirs(SELL_PLOTS_DIR, exist_ok=True)
print(f"Results will be saved in: {DATE_SPECIFIC_RESULTS_DIR}")


# Strategy Parameters
PRICE_TOLERANCE = 0.10 # 10%
TICKER_COLUMN_NAME = 'ticker'
SHORT_WINDOW = 20
MID_WINDOW = 50
LONG_WINDOW = 200


# --- 3. CORE FUNCTIONS ---

def load_ma_data(file_path, stock_name_for_print):
    """Loads and preprocesses stock data, calculating multiple SMAs for the last 5 years."""
    if not os.path.exists(file_path):
        print(f"      - WARNING: Data file not found for {stock_name_for_print} at {file_path}. Skipping.")
        return None
    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        df.index = df.index.tz_localize(None) # Make index timezone-naive
        df.sort_index(inplace=True)

        # Filter data to the last 5 years
        if not df.empty:
            five_years_ago = df.index[-1] - pd.DateOffset(years=5)
            df = df[df.index >= five_years_ago]

        # Calculate SMAs
        df[f'{SHORT_WINDOW}_SMA'] = df['close'].rolling(window=SHORT_WINDOW).mean()
        df[f'{MID_WINDOW}_SMA'] = df['close'].rolling(window=MID_WINDOW).mean()
        df[f'{LONG_WINDOW}_SMA'] = df['close'].rolling(window=LONG_WINDOW).mean()

        df.dropna(inplace=True)
        return df
    except Exception as e:
        print(f"      - ERROR: Could not process file for {stock_name_for_print}. Reason: {e}")
        return None

def plot_and_save_opportunity(df, opportunity):
    """Generates and saves a chart using the specified Matplotlib style with metadata text."""
    stock_name = opportunity['Stock']
    signal_type = opportunity['Signal Type']

    chart_df = df

    if 'BUY' in signal_type:
        metadata_text_box_color = "#b6d8a8"
        save_path = os.path.join(BUY_PLOTS_DIR, f'{stock_name}_buy_signal.png')
    else: # SELL
        metadata_text_box_color = "#f5cbcc"
        save_path = os.path.join(SELL_PLOTS_DIR, f'{stock_name}_sell_signal.png')

    # --- MODIFIED: Create the figure and axes objects to have more control ---
    fig, ax = plt.subplots(figsize=(20, 10))

    # Plotting price and SMAs with specified colors from reference image
    ax.plot(chart_df.index, chart_df['close'], color='k', lw=1, label='close')
    ax.plot(chart_df.index, chart_df[f'{SHORT_WINDOW}_SMA'], color='g', lw=1, label=f'{SHORT_WINDOW} SMA')
    ax.plot(chart_df.index, chart_df[f'{MID_WINDOW}_SMA'], color='r', lw=1, label=f'{MID_WINDOW} SMA')
    ax.plot(chart_df.index, chart_df[f'{LONG_WINDOW}_SMA'], color='k', lw=1.5, label=f'{LONG_WINDOW} SMA')

    # Plot all historical buy and sell signals from the DataFrame
    buy_signals = chart_df[chart_df['Buy_Crossover'] == 1]
    ax.plot(buy_signals.index, buy_signals['close'], '^',
             markersize=15, color='green', alpha=0.7, label='buy')

    sell_signals = chart_df[chart_df['Sell_Crossover'] == 1]
    ax.plot(sell_signals.index, sell_signals['close'], 'v',
             markersize=15, color='red', alpha=0.7, label='sell')

    ax.set_ylabel('Price in ₹', fontsize=16)
    ax.set_xlabel('Date', fontsize=16)
    ax.set_title(f"{stock_name} - SMA Crossover", fontsize=20)
    ax.legend()
    ax.grid()

    # --- ADDED: Metadata text block on the plot ---
    distance_pct = (opportunity['Current Close'] - opportunity['Crossover Price']) / opportunity['Crossover Price'] * 100
    metadata_text = (
        f"Signal Type: {opportunity['Signal Type']}\n"
        f"Crossover Date: {opportunity['Crossover Date']}\n"
        f"Crossover Price: {opportunity['Crossover Price']:.2f}\n"
        f"Current Close: {opportunity['Current Close']:.2f}\n"
        f"Distance from Crossover: {distance_pct:.2f}%"
    )

    # Add the text box to the top left of the chart
    ax.text(0.02, 0.98, metadata_text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', facecolor=metadata_text_box_color, alpha=0.5))

    try:
        fig.savefig(save_path)
        plt.close(fig) # Close the specific figure to free up memory
    except Exception as e:
        print(f"\nCould not save image for {stock_name}. Error: {e}")
        plt.close(fig)


def scan_for_ma_opportunities(df, stock_name):
    """
    Scans a single stock for opportunities using diff-based logic and refined signal classification.
    """
    # Define signal conditions for the entire DataFrame
    df['Buy_Signal'] = np.where((df[f'{SHORT_WINDOW}_SMA'] < df[f'{MID_WINDOW}_SMA']) & (df[f'{MID_WINDOW}_SMA'] < df[f'{LONG_WINDOW}_SMA']), 1.0, 0.0)
    df['Sell_Signal'] = np.where((df[f'{SHORT_WINDOW}_SMA'] > df[f'{MID_WINDOW}_SMA']) & (df[f'{MID_WINDOW}_SMA'] > df[f'{LONG_WINDOW}_SMA']), 1.0, 0.0)

    # Use diff() to find the exact crossover point
    df['Buy_Crossover'] = df['Buy_Signal'].diff()
    df['Sell_Crossover'] = df['Sell_Signal'].diff()

    # Filter for actual crossover events
    crossovers = df[(df['Buy_Crossover'] == 1) | (df['Sell_Crossover'] == 1)].copy()

    if crossovers.empty:
        return None # No crossover events found

    latest_crossover = crossovers.iloc[-1]
    last_close = df.iloc[-1]['close']
    crossover_price = latest_crossover['close']

    opp = {
        'Stock': stock_name,
        'Crossover Date': latest_crossover.name.date(),
        'Crossover Price': round(crossover_price, 2),
        'Current Close': round(last_close, 2),
    }

    # Refined logic for signal classification
    if latest_crossover['Buy_Crossover'] == 1: # The last event was a Buy Crossover
        # Condition 1: Current price is at or below the crossover price.
        if last_close <= crossover_price:
            opp['Signal Type'] = 'IMMEDIATE_BUY'
            plot_and_save_opportunity(df, opp)
            return opp
        # Condition 2: Current price is ABOVE the crossover but within the 10% tolerance.
        elif last_close > crossover_price and last_close <= crossover_price * (1 + PRICE_TOLERANCE):
            opp['Signal Type'] = 'MISSED_TRIGGER_NEARBY'
            plot_and_save_opportunity(df, opp)
            return opp

    elif latest_crossover['Sell_Crossover'] == 1: # The last event was a Sell Crossover
        # A sell signal is only relevant if the condition is still active today
        if df.iloc[-1]['Sell_Signal'] == 1:
            opp['Signal Type'] = 'SELL_SIGNAL'
            plot_and_save_opportunity(df, opp)
            return opp

    return None # No actionable opportunity found


# --- 4. MAIN EXECUTION BLOCK ---
if __name__ == '__main__':
    all_opportunities = []

    print("--- Starting MA Contrarian Daily Opportunity Scanner (V40 Stocks) ---")
    print(f"Scanning data from folder: {INPUT_DATA_PATH}")

    stock_list_path = os.path.join(DATA_BASE_PATH, SOURCE_FILE_V40)
    try:
        stocks_df = pd.read_csv(stock_list_path)
        tickers = stocks_df[TICKER_COLUMN_NAME].dropna().unique()
        print(f"  - Found {len(tickers)} unique stocks in {SOURCE_FILE_V40}.")

        for i, ticker in enumerate(tickers):
            print(f"  [{i+1}/{len(tickers)}] Analyzing: {ticker}...", end='\r')
            data_file_path = f"{INPUT_DATA_PATH}{ticker}.csv"
            df = load_ma_data(data_file_path, ticker)

            if df is not None and not df.empty:
                opportunity = scan_for_ma_opportunities(df, ticker)
                if opportunity:
                    all_opportunities.append(opportunity)

    except FileNotFoundError:
        print(f"\n  - FATAL ERROR: Stock list file not found at {stock_list_path}.")

    print("\n\n--- Scan Complete ---")

    # --- 5. FINAL REPORTING ---
    if all_opportunities:
        report_df = pd.DataFrame(all_opportunities)
        report_df['Distance From Crossover %'] = ((report_df['Current Close'] - report_df['Crossover Price']) / report_df['Crossover Price'] * 100).round(2)

        # Reorder columns
        cols_order = ['Stock', 'Signal Type', 'Current Close', 'Crossover Price',
                      'Distance From Crossover %', 'Crossover Date']
        report_df = report_df.reindex(columns=cols_order)

        # Sort for priority
        report_df['sort_key'] = report_df['Signal Type'].apply(lambda x: {'IMMEDIATE_BUY':0, 'MISSED_TRIGGER_NEARBY':1, 'SELL_SIGNAL':2}.get(x, 99))
        report_df = report_df.sort_values(by=['sort_key', 'Distance From Crossover %']).drop(columns=['sort_key'])

        output_path = os.path.join(DATE_SPECIFIC_RESULTS_DIR, OUTPUT_CSV_FILE)
        report_df.to_csv(output_path, index=False)

        print(f"\nSUCCESS: Found {len(report_df)} opportunities.")
        print("Detailed report and plots saved to:")
        print(DATE_SPECIFIC_RESULTS_DIR)
        print("\n" + "="*28 + " OPPORTUNITY SUMMARY " + "="*28)
        print(report_df.to_string())

    else:
        print("\nNo MA Contrarian opportunities found for the given date.")


Mounted at /content/drive
Results will be saved in: /content/drive/My Drive/stock_data/MA_Crossover_Result/2025-06-17
--- Starting MA Contrarian Daily Opportunity Scanner (V40 Stocks) ---
Scanning data from folder: /content/drive/My Drive/stock_data/2025-06-17/
  - Found 40 unique stocks in v40_token.csv.


--- Scan Complete ---

SUCCESS: Found 34 opportunities.
Detailed report and plots saved to:
/content/drive/My Drive/stock_data/MA_Crossover_Result/2025-06-17

============================ OPPORTUNITY SUMMARY ============================
            Stock            Signal Type  Current Close  Crossover Price  Distance From Crossover % Crossover Date
25   WHIRLPOOL-EQ          IMMEDIATE_BUY        1330.90          1595.10                     -16.56     2025-01-13
29      SANOFI-EQ          IMMEDIATE_BUY        6051.00          6765.10                     -10.56     2024-10-18
6          TCS-EQ          IMMEDIATE_BUY        3515.30          3904.50                      -9.97     2025-